# Split audio from video

## Set up

In [ ]:
!pip install ffmpeg
!pip install ffmpeg-python
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Split audio according to scences (segmentation)

In [ ]:
import os
import json
import ffmpeg
import glob

# The function to convert frames to seconds according to the corresponding fps
def convert_frame_to_second(number_frame, path_file_audio):
  info=ffmpeg.probe(path_file_audio)
  f, s = info['streams'][0]['avg_frame_rate'].split("/")
  return (number_frame*int(s))/int(f)

# Path dict_support_model
f = open("/content/drive/MyDrive/ASR_Vietnamese/dict_support_model_batch_3.json")
# Path video
FOLDER_VIDEOS = "/content/drive/MyDrive/ASR_Vietnamese/Database/Dataset_Batch3/C02_V02"
# Path audio want to save
FOLDER_AUDIOS_SPLIT = "/content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V02_batch_3"


d = json.load(f)
dict_scenes = {}
lst = []
for video in os.listdir(FOLDER_VIDEOS):
  lst = []
  for i in d.items():
    if i[1]['image_path'][26:35] == video[:-4]:
      scene = [int(i[1]['list_shot_id'][0]), int(i[1]['list_shot_id'][-1])]
      if scene not in lst:
        lst.append(scene)
  dict_scenes[f"{video[:-4]}.txt"] = lst
dict_scenes

In [ ]:
import ffmpeg, os
from tqdm import tqdm

for file_name_video in tqdm(os.listdir(FOLDER_VIDEOS)):
  path_file_name_video = f"{FOLDER_VIDEOS}/{file_name_video}"
  name_video_txt = f"{file_name_video[:-4]}.txt"

  if  name_video_txt in dict_scenes:
 
    if not os.path.isdir(f"{FOLDER_AUDIOS_SPLIT}/{file_name_video[:-4]}"):
      os.makedirs(f"{FOLDER_AUDIOS_SPLIT}/{file_name_video[:-4]}")
      print("created folder : ", f"{FOLDER_AUDIOS_SPLIT}/{file_name_video[:-4]}")

    # Get start scene and end scene of each scene
    for start_scene, end_scene in dict_scenes[name_video_txt]:
      # Convert to second
      start_time = convert_frame_to_second(start_scene,path_file_name_video)
      end_time = convert_frame_to_second(end_scene,path_file_name_video)
      stream = ffmpeg.input(path_file_name_video)

      # path save audio
      name_file_video_split = f"{FOLDER_AUDIOS_SPLIT}/{file_name_video[:-4]}/{name_video_txt[:-4]}_{start_scene}_{end_scene}.wav"
        
      audio = stream.audio.filter('atrim', start=start_time, end=end_time)
      # overwrite_output overwrites it automatically
      try:
        out, err = ffmpeg.output(audio, name_file_video_split,  **{'ar': '16000','ac':'1'}).run(capture_stderr=True)
      except ffmpeg.Error as e:
        print("output")
        print(e.stdout)
        print("err")
        print(e.stderr)


  0%|          | 0/10 [00:00<?, ?it/s]

created folder :  /content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V03_batch_3/C02_V0255


 10%|█         | 1/10 [36:13<5:25:57, 2173.06s/it]

created folder :  /content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V03_batch_3/C02_V0256


 20%|██        | 2/10 [1:09:29<4:35:52, 2069.12s/it]

...

created folder :  /content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V03_batch_3/C02_V0259


 50%|█████     | 5/10 [2:42:40<2:45:42, 1988.59s/it]

created folder :  /content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V03_batch_3/C02_V0260


 60%|██████    | 6/10 [3:11:35<2:06:50, 1902.67s/it]

created folder :  /content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V03_batch_3/C02_V0261


# Inference model ASR

## Set up

In [ ]:
!pip3 install transformers==4.9.2 soundfile datasets==1.11.0 pyctcdecode==v0.1.0
!pip3 install https://github.com/kpu/kenlm/archive/master.zip
!pip install ffmpeg
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 16.8 MB/s 
     |████████████████████████████████| 264 kB 70.1 MB/s 
     |████████████████████████████████| 880 kB 61.4 MB/s 
     |████████████████████████████████| 3.3 MB 19.0 MB/s 
     |████████████████████████████████| 212 kB 48.6 MB/s 
     |████████████████████████████████| 115 kB 26.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=3e14949aec06ce7d17c518d215759ae7d75edb8510c510b79f745c2482377fa8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 550 kB 9.9 MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2374153 sha256=6b8f730a6195ad08d0ffefc3da9dfc4c94c858

## Download model
**Source**: https://huggingface.co/nguyenvulebinh/wav2vec2-base-vietnamese-250h <br>
*Special thanks to NguyenVuLeBinh*

In [ ]:
from transformers.file_utils import cached_path, hf_bucket_url
import os, zipfile
import shutil
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf
import torch
import kenlm
from pyctcdecode import Alphabet, BeamSearchDecoderCTC, LanguageModel
import IPython

cache_dir = './cache/'
processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h", cache_dir=cache_dir)
model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h", cache_dir=cache_dir)
lm_file = hf_bucket_url("nguyenvulebinh/wav2vec2-base-vietnamese-250h", filename='vi_lm_4grams.bin.zip')
lm_file = cached_path(lm_file,cache_dir=cache_dir)
with zipfile.ZipFile(lm_file, 'r') as zip_ref:
    zip_ref.extractall(cache_dir)
lm_file = cache_dir + 'vi_lm_4grams.bin'

Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [ ]:
# Get model

def get_decoder_ngram_model(tokenizer, ngram_lm_path):
    vocab_dict = tokenizer.get_vocab()
    sort_vocab = sorted((value, key) for (key, value) in vocab_dict.items())
    vocab = [x[1] for x in sort_vocab][:-2]
    vocab_list = vocab
    # convert ctc blank character representation
    vocab_list[tokenizer.pad_token_id] = ""
    # replace special characters
    vocab_list[tokenizer.unk_token_id] = ""
    # vocab_list[tokenizer.bos_token_id] = ""
    # vocab_list[tokenizer.eos_token_id] = ""
    # convert space character representation
    vocab_list[tokenizer.word_delimiter_token_id] = " "
    # specify ctc blank char index, since conventially it is the last entry of the logit matrix
    alphabet = Alphabet.build_alphabet(vocab_list, ctc_token_idx=tokenizer.pad_token_id)
    lm_model = kenlm.Model(ngram_lm_path)
    decoder = BeamSearchDecoderCTC(alphabet,
                                   language_model=LanguageModel(lm_model))
    return decoder
# define function to read in sound file
def map_to_array(batch):
    speech, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech
    batch["sampling_rate"] = sampling_rate
    return batch

ngram_lm_model = get_decoder_ngram_model(processor.tokenizer, lm_file)

In [ ]:
from tqdm import tqdm 

# Path audio splitted
FOLDER_AUDIOS_SPLIT = "/content/drive/MyDrive/ASR_Vietnamese/AUDIOC02_V02_batch_3"

# Path trancsrip want to save
FOLDER_TRANSCRIPS = "/content/drive/MyDrive/ASR_Vietnamese/TRANSCRIPTC02_V02_batch_3"

for file_name_audios_split in tqdm(os.listdir(FOLDER_AUDIOS_SPLIT)):
  batch = {}
  path_file_name_audios_split = f"{FOLDER_AUDIOS_SPLIT}/{file_name_audios_split}"
  for file_name_audio in os.listdir(path_file_name_audios_split):
    # print(file_name_audio)

    if file_name_audio.endswith('.wav'):
      path_file_name_audio = f"{path_file_name_audios_split}/{file_name_audio}"
      batch[file_name_audio] = path_file_name_audio

  for file_name_audio, path_file_name_audio in batch.items():
    frame = file_name_audio[10:-4].split("_")
    frame_start, frame_end = int(frame[0]), int(frame[1])
    # print(frame_start, frame_end )
    
    if frame_end == frame_start:
      break
    if not os.path.isdir(f"{FOLDER_TRANSCRIPS}/{file_name_audio[:9]}"):
        os.makedirs(f"{FOLDER_TRANSCRIPS}/{file_name_audio[:9]}")
  # read in sound file
  # infer model
    try: 
      ds = map_to_array({"file": path_file_name_audio})

      input_values = processor(
              ds["speech"], 
              sampling_rate=ds["sampling_rate"], 
              return_tensors="pt"
        ).input_values
      # ).input_values.to("cuda")  # if use gpu then comment this line
      # model.to("cuda")  # if use gpu then comment this line
      logits = model(input_values).logits[0]
      pred_ids = torch.argmax(logits, dim=-1)
      output = ngram_lm_model.decode(logits.cpu().detach().numpy(), beam_width=500)
      # print(f"{file_name_audio[:9]}: {output}")
      # save transcrip
      f = open(f"{FOLDER_TRANSCRIPS}/{file_name_audio[:9]}/{file_name_audio[:-4]}.txt","w")
      f.write(output)
      f.close()
    except:
      print(f"Error: {file_name_audio}")
      # f = open(f"{FOLDER_TRANSCRIPS}/{file_name_audio[:9]}/{file_name_audio[:-4]}.txt","w")
      # output = ""
      # f.write(output)
      # f.close()

# Write csv

## Set up

In [ ]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 17.7 MB/s 
     |████████████████████████████████| 591 kB 47.9 MB/s 
     |████████████████████████████████| 235 kB 53.3 MB/s 
     |████████████████████████████████| 965 kB 44.9 MB/s 


## Write

In [ ]:
import json
f= open('/content/dict_support_model_2.json')
data= json.load(f)
data

from tqdm import tqdm
list_video_keyframes = {}
for i in tqdm(range(len(data))):
  video_keyframes = {}
  instance = data[str(i)]
  # print(instance)
  
  video = instance['image_path'][-20:-11]
  frame = instance['image_path'][-10:]
  video_keyframes['image_path'] = instance['image_path']
  shot = instance['list_shot_id']
  video_keyframes['name_video'] =  video
  video_keyframes['frame'] = frame
  video_keyframes['frame_id'] = int(frame[:-4])
  video_keyframes['frame_start_id'] = int(shot[0])
  video_keyframes['frame_end_id'] = int(shot[-1])
  video_keyframes['shot'] = shot

  list_video_keyframes[f"{video}_{video_keyframes['frame_start_id']}_{video_keyframes['frame_end_id']}"] = video_keyframes



In [ ]:
import os
import json
# Path metadata
FOLDER_METADATA = "/content/drive/MyDrive/ASR_Vietnamese/metada/Metadata_C02_V01"
meta_data = {}
for video_name in os.listdir(FOLDER_METADATA):
  f = open(f"{FOLDER_METADATA}/{video_name}")
  j_meta = json.load(f)
  # print(j_meta['keywords'])
  
  meta_data[video_name[:-5]] = j_meta['keywords']
meta_data

In [ ]:
from underthesea import classify
import csv,os
from tqdm import tqdm
FOLDER_TRANSCRIPS = "/content/drive/MyDrive/ASR_Vietnamese/TRANSCRIPTC02_V00_batch_2"
PATH_KEYFRAMES_JSON = "/content/dict_support_model_2.json"

for name_video in tqdm(os.listdir(FOLDER_TRANSCRIPS)):
  path_name_video = os.path.join(FOLDER_TRANSCRIPS,name_video)
  for transcrips in os.listdir(path_name_video):
    if transcrips[:-4] in d:
      path_transcrips_audio = os.path.join( path_name_video,transcrips)
      content = []
      # print(f"Processing: {transcrips}")

      frame = transcrips[10:-4].split("_")
      frame_start, frame_end = int(frame[0]), int(frame[1])
      # print(frame_start, frame_end)
      
      f = open(path_transcrips_audio, "r")
      transcrip = f.read()
      f.close()
      cls = classify(transcrip)
      if cls != None:
        cls = cls[0]

        # print(shot)

      content= [name_video, d[transcrips[:-4]]['shot'], transcrip, cls, meta_data[name_video]]
      # print(content)

      with open('/content/drive/MyDrive/ASR_Vietnamese/csv_C02_V01.csv', 'a', encoding='utf-8') as f:
        # create the csv writer
        writer = csv.writer(f)
        writer.writerow(content)
        # print(f"Writed: {path_transcrips_audio}")

      f.close()

